In [2]:
!python -V

Python 3.9.24


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
6
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/10/23 13:02:32 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/23 13:02:32 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1761135095530, experiment_id='1', last_update_time=1761135095530, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

/tmp/ipykernel_27745/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict on validation set
y_pred = lr.predict(X_val)

# Compute RMSE manually
mse = mean_squared_error(y_val, y_pred)  # returns MSE
rmse = np.sqrt(mse)                       # take square root for RMSE
print(f'Val RMSE: {rmse}')


Val RMSE: 7.758715209663881


In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    # Log data paths
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    # Model parameters
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    # Train model
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    # Predict
    y_pred = lr.predict(X_val)

    # Compute RMSE manually (compatible with any sklearn version)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [41]:
from sklearn.metrics import mean_squared_error
import numpy as np
import mlflow
from hyperopt import STATUS_OK

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        # Train XGBoost
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        # Predict on validation set
        y_pred = booster.predict(valid)

        # Compute RMSE manually
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))  # safer than squared=False
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}


In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:37:38] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.42848                           
[1]	validation-rmse:7.12798                           
[2]	validation-rmse:6.70993                           
[3]	validation-rmse:6.56238                           
[4]	validation-rmse:6.50095                           
[5]	validation-rmse:6.46590                           
[6]	validation-rmse:6.44980                           
[7]	validation-rmse:6.44326                           
[8]	validation-rmse:6.43486                           
[9]	validation-rmse:6.42829                           
[10]	validation-rmse:6.42329                          
[11]	validation-rmse:6.41826                          
[12]	validation-rmse:6.41545                          
[13]	validation-rmse:6.41088                          
[14]	validation-rmse:6.40431                          
[15]	validation-rmse:6.40041                          
[16]	validation-rmse:6.39592                          
[17]	validation-rmse:6.39430                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:38:32] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.73409                                                   
[1]	validation-rmse:11.29252                                                   
[2]	validation-rmse:10.88584                                                   
[3]	validation-rmse:10.51192                                                   
[4]	validation-rmse:10.16922                                                   
[5]	validation-rmse:9.85454                                                    
[6]	validation-rmse:9.56701                                                    
[7]	validation-rmse:9.30342                                                    
[8]	validation-rmse:9.06403                                                    
[9]	validation-rmse:8.84490                                                    
[10]	validation-rmse:8.64562                                                   
[11]	validation-rmse:8.46510                                                   
[12]	validation-rmse:8.29994            

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:40:44] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.24711                                                     
[1]	validation-rmse:10.43005                                                     
[2]	validation-rmse:9.74257                                                      
[3]	validation-rmse:9.16905                                                      
[4]	validation-rmse:8.68934                                                      
[5]	validation-rmse:8.29353                                                      
[6]	validation-rmse:7.96759                                                      
[7]	validation-rmse:7.70095                                                      
[8]	validation-rmse:7.48333                                                      
[9]	validation-rmse:7.30586                                                      
[10]	validation-rmse:7.15983                                                     
[11]	validation-rmse:7.04106                                                     
[12]	validation-

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:44:36] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.69470                                                       
[1]	validation-rmse:8.27328                                                       
[2]	validation-rmse:7.50653                                                       
[3]	validation-rmse:7.09190                                                       
[4]	validation-rmse:6.87382                                                       
[5]	validation-rmse:6.75513                                                       
[6]	validation-rmse:6.67767                                                       
[7]	validation-rmse:6.63185                                                       
[8]	validation-rmse:6.60739                                                       
[9]	validation-rmse:6.59017                                                       
[10]	validation-rmse:6.57919                                                      
[11]	validation-rmse:6.56999                                                      
[12]

KeyboardInterrupt: 

In [43]:
mlflow.xgboost.autolog(disable=True)

In [44]:
import mlflow
import mlflow.xgboost
import xgboost as xgb
import pickle
from sklearn.metrics import mean_squared_error
import numpy as np

with mlflow.start_run():
    
    # Prepare DMatrix
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',  # updated
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    # Train model
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    # Predict
    y_pred = booster.predict(valid)

    # Compute RMSE manually (works for all sklearn versions)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    # Save and log preprocessor
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    # Log the XGBoost model
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

2025/10/23 12:48:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:48:44] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/10/23 12:48:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [1]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
import numpy as np
import mlflow
import mlflow.sklearn

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        # Log data paths and artifacts
        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        # Train model
        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        # Predict on validation set
        y_pred = mlmodel.predict(X_val)

        # Compute RMSE manually (avoids squared=False issue)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        


MlflowException: Could not find experiment with ID 0